In [ ]:
import arcpy
import geojson
import json
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import os
import time
import numpy as np
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from pathlib import Path
import pathlib


PLANET_API_KEY=''

orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}

# Define the path to your geodatabase and feature class
gdb_path = 'C:\\Users\\msmurphy\\Documents\\WinterTurf\\projects\\DEM WinterTurf\\DEM WinterTurf\\WinterTurf_Greens_21_24.gdb'
input_excel_courses = "C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\Wausau 2025 input.xlsx"

#Directory paths
bDir = 'C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\'
iDir = bDir + 'json'
oDir = bDir + 'images'


#output variables
order_nickname = "Wausau 2021 PSBSD"
output_file_path = "C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\Wausau 2021 PSBSD.csv"

print(datetime.now())


2025-08-15 21:34:15.673118


In [2]:
#some json functions

def open_geojson(file_path):
    with open(file_path) as f:
#         gj = geojson.load(f)
        gj = json.load(f)
        
    return gj

def get_geojson_geometry(geojson):
    geometry = [i['geometry'] for i in geojson['features']]
    
    return geometry

def plot_wrapper(gdf, ax, color, linewidth=1.5):
    '''Convenience function for overlaying spatial data on the same plot'''
    gdf.plot(
        facecolor="none",
        edgecolor=color,
        linewidth=linewidth,
        ax = ax   
    )

# define helpful functions for submitting, polling, and downloading an order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

# print(datetime.now())

# Function to retrieve all features
def get_all_features(search_params, auth):
    features = []
    offset = 0
    while True:
        search_params['offset'] = offset
        response = requests.get(orders_url, headers=headers, auth=auth, params=search_params)
        response.raise_for_status()  # Raise an error for bad requests
        data = response.json()
        # print(data)
        features.extend(data['features'])
        if len(data['features']) < 250:
            break
        offset += 250
    return features

print(datetime.now())

2025-08-15 21:24:19.994803


In [9]:
file_path = input_excel_courses
df_courses = pd.read_excel(file_path)

#create a dataframe for the output

# df_output = pd.DataFrame(columns=['Course','Order_2017','Order_2018','Order_2019','Order_2020','Order_2021','Order_2022','Order_2023', 'Order_2024', 'Order_2025'])
df_output = pd.DataFrame(columns=['Course','Order_2021'])
df_output['Course'] = df_courses['Course']

for index, row in df_courses.iterrows(): 
    course = row['Course']
    feature_class_name = course
    # years = ['2025']
    years = ['2021']
    for year in years:
        start_date = year+'-01-01'
        end_date = year+'-12-31'

        if not os.path.exists(iDir+"\\"+feature_class_name+".geojson"):
            arcpy.conversion.FeaturesToJSON(
                in_features=gdb_path+"\\"+feature_class_name,
                out_json_file=iDir+"\\"+feature_class_name+".geojson",
                format_json="NOT_FORMATTED",
                include_z_values="NO_Z_VALUES",
                include_m_values="NO_M_VALUES",
                geoJSON="GEOJSON",
                outputToWGS84="WGS84",
                use_field_alias="USE_FIELD_NAME"
            )

        geojson_file = iDir+"\\"+feature_class_name+".geojson"

        # Read the GeoJSON file into a GeoDataFrame for the aoi
        gdf_aoi = gpd.read_file(geojson_file)

        #Create the geometry for the Planet query
        boundary_geojson = open_geojson(geojson_file)

        bounds = get_geojson_geometry(boundary_geojson)

        geojson_geometry = bounds[0]

        #Set up filters
        geometry_filter = {
        "type": "GeometryFilter",
        "field_name": "geometry",
        "config": geojson_geometry
        }

        instrument_filter = {
        "type":"StringInFilter",
        "field_name":"instrument",
        "config":[
            "PSB.SD"
            # "PS2.SD"
            # "PS2"
        ]
        }

        quality_category_filter = {
        "type":"StringInFilter",
        "field_name":"quality_category",
        "config":[
            "standard"
        ]
        }

        date_range_filter = {
        "type": "DateRangeFilter",
        "field_name": "acquired",
        "config": {
            "gte": start_date+"T00:00:00.000Z",
            "lte": end_date+"T00:00:00.000Z"
        }
        }

        # only get images which have <5% cloud coverage
        cloud_cover_filter = {
        "type": "RangeFilter",
        "field_name": "cloud_cover",
        "config": {
            "lte": 0.05
        }
        }

        # orthoanalytic images that have 4 bands and are corrected for surface reflectance
        asset_filter = {
        "type": "AssetFilter",
        "config": ["ortho_analytic_4b_sr"]
        }

        # combine filters including clear filters
        was_combined_filter = {
        "type": "AndFilter",
        "config": [geometry_filter, date_range_filter, quality_category_filter, cloud_cover_filter, asset_filter, instrument_filter]
        }

        item_type = "PSScene"

        # API request object
        search_request = {
        "item_types": [item_type], 
        # "limit":250,
        # "offset": 0,
        "filter": was_combined_filter
        }

        # # fire off the POST request
        search_result = \
        requests.post(
            'https://api.planet.com/data/v1/quick-search?_sort=acquired asc&_page_size=250',
            auth=HTTPBasicAuth(PLANET_API_KEY, ''),
            json=search_request)

        # print(search_result.json()['_links'])
        try:
            next_url = search_result.json()['_links']['_next']
        except:
            next_url = None

        next_ids = []
        image_ids = []

        if next_url is not None:
            next = requests.get(next_url, auth=auth)
            next_ids = [feature['id'] for feature in next.json()['features']]
            output_features = search_result.json()['features'] + next.json()['features']
        else:
            output_features = search_result.json()['features']
        

        image_ids = [feature['id'] for feature in search_result.json()['features']]
        
        image_ids = image_ids + next_ids        

        image_ids.sort()
        print(course, year, ' total number of images:', len(image_ids))

        #Create a dataframe out of the json response to the search
        # df_tiles = pd.json_normalize(search_result.json()['features'], max_level=1)
        df_tiles = pd.json_normalize(output_features, max_level=1)
        # print(len(df_tiles))

        # rename properties column, dropping the properties label
        prop_cols = {col: col.split('.')[1] for col in df_tiles.columns if 'properties' in col}
        df_tiles.rename(columns=prop_cols, inplace=True)

        geoms = [Polygon(geo[0]) for geo in df_tiles['geometry.coordinates']]
        gdf_tiles = gpd.GeoDataFrame(df_tiles, geometry = geoms, crs=4326)

        gdf_tiles_contain_aoi = gpd.sjoin(gdf_tiles, gdf_aoi,  how='inner', predicate='contains' )

        gdf_tiles_contain_aoi['date_acquired'] = pd.to_datetime(gdf_tiles_contain_aoi['acquired'].str[:10])

        #sort by clear percentage
        try:
            # df_tiles_sorted = gdf_tiles_contain_aoi.sort_values(by='clear_percent', ascending=False)
            df_tiles_sorted = gdf_tiles_contain_aoi.sort_values(by='clear_confidence_percent', ascending=False)
            # Drop duplicates and keep the first occurrence of each date (highest clear percentage)
            df_tiles_unique = df_tiles_sorted.drop_duplicates(subset='date_acquired', keep='first')
        except:
            print("Couldn't sort by clear confidence percentage")
            df_tiles_unique = gdf_tiles_contain_aoi.drop_duplicates(subset='date_acquired', keep='first')


        df_tiles_unique.to_csv("C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\"+course+"_"+year+"PSBSD_downloaded.csv")


        ############### Check if we already have any of the tiles we want to order ###################

        # Set the workspace to your geodatabase
        arcpy.env.workspace = "C:\\Users\\msmurphy\\Documents\\ArcGIS\\Projects\\Greens\\"+course+".gdb"

        # List all rasters in the geodatabase
        rasters = arcpy.ListRasters()

        # Filter rasters that end with "3B_AnalyticMS_SR_clip"
        filtered_rasters = [raster for raster in rasters if raster.endswith("3B_AnalyticMS_SR_clip")]

        # Extract the beginning of the raster names
        acquired_tiles = [raster[1:-22] for raster in filtered_rasters]

        filtered_list = df_tiles_unique['id'].to_list()

        # Print the list
        print("*************** Tiles we want ***************")
        print(filtered_list)
        print()

        common_values = list(set(acquired_tiles) & set(filtered_list)) 
        print("*************** Tiles we already have ***************")
        print(common_values)
        print()

        order_list = [item for item in filtered_list if item not in acquired_tiles] 
        print("*************** Order list ***************")
        print(order_list)

        #if you are not going to filter by tiles we already have
        #order_list = df_tiles_unique['id'].to_list()

        num_tiles = len(order_list)

        print(course+": ordering "+str(len(order_list))+" tiles.")

        course_product = [
        {
        "item_ids": order_list,
        "item_type": "PSScene",
        "product_bundle": "analytic_sr_udm2"
        }
        ]

        # define the clip tool
        clip = {
            "clip": {
                "aoi": geojson_geometry
            }
        }

        # create an order request with the clipping tool
        request_clip = {
        "name": order_nickname+" "+course+" "+year,
        "products": course_product,
        "tools": [clip]
        }


        clip_order_url = place_order(request_clip, auth)

        df_output.loc[df_output['Course'] == course, 'Order_'+year] = clip_order_url

        print("finished "+course+" "+str(datetime.now()))

df_output.to_csv(output_file_path)

Course_02I_Outline 2021  total number of images: 174
*************** Tiles we want ***************
['20210614_160521_77_245a', '20210815_164912_42_2408', '20210705_160303_71_2212', '20210422_160149_05_2444', '20210901_161231_97_225a', '20210618_161110_47_2280', '20210616_164930_56_240a', '20210320_165506_51_2406', '20210610_165346_86_2406', '20210605_161226_56_2441', '20210604_160251_85_2448', '20210529_160352_40_2430', '20210430_161444_51_2233', '20210823_160028_67_2450', '20210702_160155_38_2447', '20210913_155839_92_2428', '20210928_165012_47_2405', '20210814_160335_07_242a', '20210711_165426_75_240c', '20210929_160306_13_2440', '20210923_160138_18_2235', '20210919_160101_32_245c', '20210918_165250_42_227c', '20210816_160107_80_2420', '20210612_155946_11_2460', '20210701_160332_50_2432', '20210531_161216_02_2280', '20210518_160549_49_2436', '20210724_160414_51_2436', '20210813_155925_42_242b', '20210407_160119_57_2458', '20210906_160539_86_220b', '20210417_165619_65_225b', '20210319

In [ ]:
#if the above fails, output the partial df of order numbers
df_output.to_csv(output_file_path)